# ISW preprocessing

### Install packages

In [1]:
%pip install nltk num2words scikit-learn pandas numpy

Note: you may need to restart the kernel to use updated packages.


### Import and download all dependecies

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from num2words import num2words
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import string
from zipfile import ZipFile, ZipInfo
from pathlib import Path
import os

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lap2r\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lap2r\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lap2r\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lap2r\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Define preprocessing functions

In [3]:
# Functions
def to_lower_case(text):
  return "".join([i.lower() for i in text])

stop_punctuation = string.punctuation
def remove_punctuation(text):
  return "".join([i for i in text if i not in stop_punctuation])

def remove_long_dash(text):
  return re.sub(r'—', ' ', text)

def remove_urls(text):
  return re.sub(r'http\S+', '', text)

def remove_one_letter_words(tokens):
  return list(filter(lambda token: len(token) > 1, tokens))

def tokenize_text(text):
  return nltk.tokenize.word_tokenize(text)

stop_words = set(nltk.corpus.stopwords.words('english'))
frequent_words = {'russian', 'force', 'ukrainian', 'oblast' 'ukraine', 'military', 'reported', 'effort', 'likely',
                  'claimed', 'russia', 'area', 'operation', 'continued', 'city', 'general', 'near', 'attack',
                  'official', 'staff', 'also', 'stated', 'source', 'oblast', 'pm', 'am', 'ukraine'}
month_names = {'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december'}
# avoid_stop_words = {"not", "n't", "no"}
# stop_words = stop_words - avoid_stop_words
stop_words = stop_words.union(frequent_words)
stop_words = stop_words.union(month_names)

def remove_stop_words(tokens):
  return [i for i in tokens if i not in stop_words]

def do_stemming(tokens):
  ps = nltk.PorterStemmer()
  return [ps.stem(word) for word in tokens]

def do_lemmatization(tokens):
  wn = nltk.WordNetLemmatizer()
  return [wn.lemmatize(word) for word in tokens]

def remove_numeric_words(text):
  return re.sub(r'\S*\d+\S*', '', text)

def convert_nums_to_words(data):
  tokens = data
  new_text = []
  for word in tokens:
    if word.isdigit():
      if int(word)<1000000000:
        word = num2words(word)
      else: 
        word = ""
    new_text.extend(tokenize_text(re.sub("(-|,\s?)|\s+", " ", word)))
  return new_text

def do_preprocessing(data):
  text_clean = data
  text_clean = remove_urls(text_clean)
  text_clean = remove_punctuation(text_clean)
  text_clean = remove_long_dash(text_clean)
  text_clean = to_lower_case(text_clean)
  text_clean = remove_numeric_words(text_clean)
  words = tokenize_text(text_clean)
  words = remove_one_letter_words(words)
  words = remove_stop_words(words)
  lemmatized = do_lemmatization(words)
  res = convert_nums_to_words(lemmatized)
  return res

### Zip opening

In [4]:
# specifying the zip file name
file_name = "../input_data/isw/isw_scrapping_res.zip"

df = pd.DataFrame(columns = ["Name", "Date", "Text"])
print("Openning zip in read mode")
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zipfile:
      for file in zipfile.infolist():
        if not ZipInfo.is_dir(file):
          filename = file.filename.rsplit('/', 1)[1].split('.')[0]
          date = filename.replace("assessment-", "")
          text = zipfile.read(file.filename).decode('utf-8')
          df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)

C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)
C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)
C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)
C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be remove

Openning zip in read mode


C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)
C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)
C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": filename, "Date": date, "Text": text}, ignore_index = True)
C:\Users\lap2r\AppData\Local\Temp\ipykernel_23792\1405460261.py:13: FutureWarning: The frame.append method is deprecated and will be remove

### TF-IDF creation

In [5]:
print("Find tokens")          
df["Tokens"] = df["Text"].apply(lambda d: " ".join(do_preprocessing(d)))


# print("Find most common words")
# all_words = []
# for tokens in df["Tokens"]:
#   for word in tokens.split(" "):
#     all_words.append(word)
# all_words = nltk.FreqDist(all_words)
# print("Top 30 frequenty used words: ")
# print(all_words.most_common(30))


filenames = df["Name"]
dates = df["Date"]

print("Create vectors")
tfidf = TfidfVectorizer(smooth_idf=True,use_idf=True)
vectors = tfidf.fit_transform(df["Tokens"])

# store content
with open("results/tfidf.pkl", "wb") as handle:
  pickle.dump(tfidf, handle)

feature_names = tfidf.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
dictionaries = df.to_dict(orient='records')

print("Into result")
res = __builtins__.zip(filenames, dates, dictionaries)
res_df = pd.DataFrame(res, columns=["Name","Date","Keywords"])
res_df["Keywords"] = res_df["Keywords"].apply(lambda d: {k: v for k, v in d.items() if v > 0})
res_df["Keywords"] = res_df["Keywords"].apply(lambda d: dict(sorted(d.items(), key=lambda item: item[1], reverse=True)))

res_df

Find tokens
Create vectors
Into result


,Name,Date,Keywords
0,assessment-2023-03-01,2023-03-01,"{'vučić': 0.2388396694907684, 'serbia': 0.2094..."
1,assessment-2023-03-02,2023-03-02,"{'force': 0.283954673018682, 'bakhmut': 0.1564..."
2,assessment-2023-03-03,2023-03-03,"{'force': 0.21761787610577515, 'election': 0.2..."
3,assessment-2023-03-04,2023-03-04,"{'force': 0.359067916875876, 'kartapalov': 0.1..."
4,assessment-2023-03-05,2023-03-05,"{'force': 0.446364675680866, 'bakhmut': 0.3082..."
...,...,...,...
388,assessment-2023-02-24,2023-02-24,"{'russia': 0.2811601920229502, 'force': 0.2696..."
389,assessment-2023-02-25,2023-02-25,"{'force': 0.28165989152524007, 'china': 0.2245..."
390,assessment-2023-02-26,2023-02-26,"{'putin': 0.2795991919134534, 'ultranationalis..."
391,assessment-2023-02-27,2023-02-27,"{'force': 0.31688852149145263, 'manual': 0.172..."


### Forming zip with .csv output

In [6]:
filename = "tfidf-result"
# compression_options = dict(method='zip', archive_name=f'{filename}.csv')
# res_df.to_csv(f'results/{filename}.zip', compression=compression_options, index=False)